In [ ]:
import pickle
import numpy as np
from time import time

In [ ]:
from math import sqrt
from numpy import dot
from numpy.linalg import norm
from math import log10

def cos_sim(a, b): 
    return dot(a, b) / (norm(a) * norm(b))

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec.load("data\\w2v_model")

In [ ]:
import pickle
import os
import re
import nltk
import glob
import pandas as pd

from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)

stops = set(stopwords.words("english"))

In [ ]:
def SplitTextIntoSentences(txt):
    dtxt=re.sub(r"\.\s|\.\n",".天官赐福,百无禁忌%@!^&ЁЪЫї",txt)
    etxt=re.sub(r"\!\s|\!\n","!天官赐福,百无禁忌%@!^&ЁЪЫї",dtxt)
    qtxt=re.sub(r"\?\s|\?\n","?天官赐福,百无禁忌%@!^&ЁЪЫї",etxt)
    sentences= re.split(r'\天\官\赐\福\,\百\无\禁\忌\%\@\!\^\&\Ё\Ъ\Ы\ї', qtxt)

    return sentences

In [ ]:
email = '[a-zA-Z0-9]+@\w+[\.\w+]+'
money='\$[0-9]+[\.[0-9]+]?[k]?'
date='[0-9]{1,2}\s[a-zA-Z]+\s[0-9]{1,4}|[0-9]{1,2}\s[0-9]{1,2}\s[0-9]{1,4}|[0-9]{1,2}\S[0-9]{1,2}\S[0-9]{1,4}|[a-zA-Z]+\s[0-9]{1,2}\S?\s[0-9]{1,4}'
phone='\([0-9]{3}\)\s[0-9]{3}\-[0-9]{4}'
words='[a-zA-z]+[\'[a-z]+]?|\S'
prop_names=''
full_exp=email+'|'+money+'|'+date+'|'+phone+'|'+words



tokenize_regex = re.compile(full_exp, re.I)

In [ ]:
def w2vec_vectorize(text: str, model):
    sentence_list = SplitTextIntoSentences(text)
    sent_vectors = []
    for sentence in sentence_list:
        tokens=tokenize_regex.findall(sentence)
        word_vectors = []
        for token in tokens:
            if token not in stops:
                try:
                    word_vectors.append(model.wv[token.lower()])
                except Exception as e:
                    print(e)

        sent_vector = np.zeros(model.vector_size)
        if (len(word_vectors) > 0):
            sent_vector = (np.array([sum(x) for x in zip(*word_vectors)])) / sent_vector.size
        sent_vectors.append(sent_vector)

    vector = np.zeros(model.vector_size)
    if (len(sent_vectors) > 0):
        vector = (np.array([sum(x) for x in zip(*sent_vectors)])) / vector.size

    return vector

In [ ]:
#custom_vec

dictionary=dict()

In [ ]:
with open('/content/drive/MyDrive/NLP/short/data/dictionary.pkl', "rb") as fp:
        dictionary = pickle.load(fp)

In [ ]:
td_matrix=dict()

with open('data\\td_matrix.pkl', "rb") as fp:
        td_matrix = pickle.load(fp)

In [ ]:
td=dict()

with open('data\\td.pkl', "rb") as fp:
        td = pickle.load(fp)

In [ ]:
def customTextVectorization(txt):
    sentences=SplitTextIntoSentences(txt)
    tf_matrix=pd.DataFrame(columns=dictionary.keys())
    for s in sentences:
              tokens=tokenize_regex.findall(s)
              tf_matrix.loc[s]=0
              for token in list(dictionary):
                if token in tokens:
                    tf_matrix.at[s,token]+=1
    tf_idf_matrix=tf_matrix

    N = len(td_matrix)

    for index, row in tf_idf_matrix.iterrows():
          for token in dictionary:
                tf_idf_matrix.at[index, token] = row[token] * log10((N+1)/(td.get(token,0)+1))

    doc_vect=[]

    for token in dictionary:
        doc_vect.append(round(tf_idf_matrix[token].mean(), 3))

    with open("data\\fitted_pca", "rb") as file:
      fitted_pca = pickle.load(file)
    vector = np.array(doc_vect).reshape(1, -1)
    doc_vect = fitted_pca.transform(vector)[0]

    return doc_vect
    

In [ ]:
with open('D:\\S\\NLP\\nlp-22-autumn\\projects\\shiyan_v3\\assets\\date\\train\\alt.atheism\\49960','r',encoding='utf-8',errors='ignore') as f:
  text1 = f.read().rstrip()
with open('D:\\S\\NLP\\nlp-22-autumn\\projects\\shiyan_v3\\assets\\date\\train\\alt.atheism\\51060','r',encoding='utf-8',errors='ignore') as f:
    text2 = f.read().rstrip()

sentences1 = text1
sentences2 = text2

In [ ]:
t0 = time()
custom1 = customTextVectorization(text1)
custom2 = customTextVectorization(text2)
t1 = time()
custom_mean_time = round((t1 - t0) / 2, 2)

t0 = time()
w2vec1 = w2vec_vectorize(text1,model)
w2vec2 = w2vec_vectorize(text2,model)
t1 = time()
w2vec_mean_time = round((t1 - t0) / 2, 2)

custom_sim = cos_sim(custom1, custom2)
w2vec_sim = cos_sim(w2vec1, w2vec2)

print(f'time: {custom_mean_time} sec %% Custom cosine similarity is {custom_sim}')
print(f'time: {w2vec_mean_time} sec %% Word2vec cosine similarity is {w2vec_sim}\n')

In [ ]:
text3='Chapter 114: Shut Up and Do The Impossible. The gibbous moon riding higher in the cloudless sky, the stars and wash of the Milky Way visible in all their majesty within the darkness, all these shone down upon the graveyard to bear witness from their unimaginable distances. In the instant when Harry had realised there was no way at all left to save everyone, his mind\'s voices had fallen away, become one, a single purpose taking up every fraction of his mind. Fifty seconds. Forty seconds. Harry\'s eyes tracked slowly across the air, until his gaze landed on the first Death Eater, the one closest to him. Thirty seconds? Twenty seconds? Time\'ss almosst up - hissed Voldemort.'
text4='Chapter 115: Shut Up and Do The Impossible, Pt 2. Something like a fugue state had come over Harry\'s mind. The absolute state had partially worn off him, partially stayed with him. Elements of his mind were numb, maybe deliberately numbed by some part that was smart enough to predict what would happen otherwise. What he\'d just done -The thought was shut off, making space for an awareness of other things. Harry was standing in the middle of a haphazard graveyard, tombstones scattered without order. By moonlight and starlight, it could be seen that black robes littered the ground, surrounded by textures that didn\'t match the surrounding graveyard earth, wetness tinged red in the moonlight. Some heads had come loose from the surrounding hoods of the robes, revealing hair that was long or short, dark or bright, which was all that could be seen beneath the moon. The silver masks stayed on, making all the hair originate in skulls instead of human faces -The thought was shut off, making space for awareness of other things.'

t0 = time()
custom1 = customTextVectorization(text3)
custom2 = customTextVectorization(text4)
t1 = time()
custom_mean_time = round((t1 - t0) / 2, 2)

t0 = time()
w2vec1 = w2vec_vectorize(text3,model)
w2vec2 = w2vec_vectorize(text4,model)
t1 = time()
w2vec_mean_time = round((t1 - t0) / 2, 2)

custom_sim = cos_sim(custom1, custom2)
w2vec_sim = cos_sim(w2vec1, w2vec2)

print(f'time: {custom_mean_time} sec %% Custom cosine similarity is {custom_sim}')
print(f'time: {w2vec_mean_time} sec %% Word2vec cosine similarity is {w2vec_sim}\n')

In [ ]:
text5='We evaluated our data using various classification and object detection architectures mentioned in Section 4 to establish the requirement of a data in non-controlled settings.'
text6='We estimated our data using different classification and object detection architectures mentioned in Section 4 to establish the requirement of a data in non-controlled settings.'

t0 = time()
custom1 = customTextVectorization(text5)
custom2 = customTextVectorization(text6)
t1 = time()
custom_mean_time = round((t1 - t0) / 2, 2)

t0 = time()
w2vec1 = w2vec_vectorize(text5,model)
w2vec2 = w2vec_vectorize(text6,model)
t1 = time()
w2vec_mean_time = round((t1 - t0) / 2, 2)

custom_sim = cos_sim(custom1, custom2)
w2vec_sim = cos_sim(w2vec1, w2vec2)

print(f'time: {custom_mean_time} sec %% Custom cosine similarity is {custom_sim}')
print(f'time: {w2vec_mean_time} sec %% Word2vec cosine similarity is {w2vec_sim}\n')

In [ ]:
len(text6)